In [1]:
import cv2
import os
import mediapipe as mp
import pandas as pd
from utilities import *
from joint_angles import JointAngle
from body_parts import BodyPart

In [ ]:
n = 8
angles = [0] * n
bp = BodyPart()

In [ ]:
mpPose = mp.solutions.pose
video = '../assets/sample-video/sample-video05.mp4'
cap = cv2.VideoCapture(video)

In [ ]:
data = {
    'Angles': [],
    'Phase': [],
    'Score': []
}

In [ ]:
output_folder = '../assets/sample-frames/sample-video05/'
os.makedirs(output_folder, exist_ok=True)
frameCnt = 0

with mpPose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.75) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        frameName = f'frame_{frameCnt}.jpg'
        framePath = os.path.join(output_folder,frameName)
        cv2.imwrite(framePath, image)
        frameCnt += 1

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True
        
        try:
            lm = results.pose_landmarks.landmark
            jointAngle = JointAngle(lm)
            bp.cords = detect_joints(lm)
            angles = jointAngle.body_angles(bp)
            data['Angles'].append(angles)
            data['Phase'].append(None)
            data['Score'].append(None)
        except:
            pass

In [ ]:
df = pd.DataFrame(data)
df
df.to_csv('../dataset/angles05.csv')

In [ ]:
df

In [ ]:
PhaseName = {
    'Down': 1,
    'Null': 0,
    'Up': 2
}

fileName = '../dataset/angles.csv'
df = pd.read_csv(fileName)

for i in range(len(df['Phase'])):
    df['Phase'][i] = PhaseName[df['Phase'][i]]

In [14]:
df = df.drop('Unnamed: 0', axis=1)
df.to_csv('../dataset/angles_final.csv')